In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn, optim

from torchvision import datasets, transforms, models


In [ ]:
# /content/drive/MyDrive/monkeyFox

In [ ]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

In [ ]:
dataset = datasets.ImageFolder('/content/drive/MyDrive/monkeyFox/original', transform=transform)

In [ ]:
total_count = len(dataset) 
total_count

2985

In [ ]:
dataset[0]

(tensor([[[0.1333, 0.1333, 0.1333,  ..., 0.1294, 0.1294, 0.1255],
          [0.1333, 0.1333, 0.1333,  ..., 0.1294, 0.1294, 0.1255],
          [0.1255, 0.1255, 0.1255,  ..., 0.1294, 0.1294, 0.1255],
          ...,
          [0.2039, 0.2196, 0.2745,  ..., 0.0353, 0.0314, 0.0275],
          [0.2039, 0.2000, 0.2588,  ..., 0.0353, 0.0314, 0.0275],
          [0.2118, 0.1843, 0.2314,  ..., 0.0431, 0.0431, 0.0392]],
 
         [[0.0941, 0.0941, 0.0941,  ..., 0.0941, 0.0941, 0.0941],
          [0.0941, 0.0941, 0.0941,  ..., 0.0941, 0.0941, 0.0941],
          [0.0941, 0.0980, 0.0980,  ..., 0.0941, 0.0941, 0.0941],
          ...,
          [0.1333, 0.1294, 0.1725,  ..., 0.0314, 0.0314, 0.0314],
          [0.1333, 0.1137, 0.1569,  ..., 0.0314, 0.0314, 0.0314],
          [0.1412, 0.1098, 0.1490,  ..., 0.0392, 0.0392, 0.0353]],
 
         [[0.0980, 0.0980, 0.0980,  ..., 0.1059, 0.1059, 0.1059],
          [0.0980, 0.0980, 0.0980,  ..., 0.1059, 0.1059, 0.1059],
          [0.0941, 0.0980, 0.0980,  ...,

In [ ]:
train_count = int(0.7 * total_count)
valid_count = int(0.2 * total_count)
test_count = total_count - train_count - valid_count
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_count, valid_count, test_count)
)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
                       
epoches = 3
steps = 0

train_losses, test_losses = [], []

for e in range(epoches):
    running_loss = 0
    
    for images, labels in train_dataloader:
        steps += 1
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logits = model(images)
        
        loss = criterion(logits, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        train_losses.append(running_loss)
        
        if steps % 20 == 0:
            test_loss, accuracy = 0, 0
        
            with torch.no_grad():
                model.eval()

                for images, labels, in valid_dataloader:
                    images, labels = images.to(device), labels.to(device)

                    logits = model(images)

                    test_loss += criterion(logits, labels)

                    ps = torch.exp(logits)

                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)

                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()


            print(f"Epoch: {e+1}/{epoches};"
                  f"Train_loss: {running_loss};"
                  f"Test_loss: {test_loss/len(valid_dataloader)};"
                  f"Accuracy: {accuracy/len(valid_dataloader)}")
            model.train()
            running_loss = 0

Epoch: 1/3;Train_loss: 26.938726484775543;Test_loss: 0.8884866833686829;Accuracy: 0.691416040847176
Epoch: 1/3;Train_loss: 14.53879901766777;Test_loss: 0.5725051760673523;Accuracy: 0.7877506268651862
Epoch: 1/3;Train_loss: 13.6056849360466;Test_loss: 0.5270909667015076;Accuracy: 0.8214285718767267
Epoch: 2/3;Train_loss: 7.3135859072208405;Test_loss: 0.43482041358947754;Accuracy: 0.8526785718767267
Epoch: 2/3;Train_loss: 9.93265414237976;Test_loss: 0.38753220438957214;Accuracy: 0.8576127824030424
Epoch: 2/3;Train_loss: 10.758047074079514;Test_loss: 0.4579276144504547;Accuracy: 0.8262844618998075
Epoch: 3/3;Train_loss: 4.344594478607178;Test_loss: 0.4368804097175598;Accuracy: 0.8205670432040566
Epoch: 3/3;Train_loss: 7.977914527058601;Test_loss: 0.39905136823654175;Accuracy: 0.8395989976431194
Epoch: 3/3;Train_loss: 7.95258004963398;Test_loss: 0.38473108410835266;Accuracy: 0.841948622151425


In [ ]:
test_loss, accuracy = 0, 0
with torch.no_grad():
    model.eval()

    for images, labels, in test_dataloader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)

        test_loss += criterion(logits, labels)

        ps = torch.exp(logits)

        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)

        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test_loss: {test_loss/len(test_dataloader)};"
           f"Accuracy: {accuracy/len(test_dataloader)}")

Test_loss: 0.3759746849536896;Accuracy: 0.8721590936183929
